In [ ]:
!pip install flask-ngrok opencv-python-headless

In [ ]:
!ngrok config add-authtoken 2vKVPhBxZgZ3y8sQ4HOCpsAThoD_31T6SyJ57zem9f9nmuAtc

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install pyngrok
!pip install ultralytics

In [ ]:
!git clone https://github.com/Dt-Pham/Advanced-Lane-Lines.git
%cd Advanced-Lane-Lines/

Cloning into 'Advanced-Lane-Lines'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 104 (delta 3), reused 0 (delta 0), pack-reused 99 (from 1)
Receiving objects: 100% (104/104), 95.77 MiB | 24.91 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/Advanced-Lane-Lines/Advanced-Lane-Lines/Advanced-Lane-Lines/Advanced-Lane-Lines/Advanced-Lane-Lines


In [ ]:
# Manually install ngrok
!wget -q -c https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
# 🔥 Start ngrok tunnel in background
!./ngrok http 5000 &

Usage of ngrok requires a verified account and authtoken.

Sign up for an account: https://dashboard.ngrok.com/signup
Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken

ERR_NGROK_4018



In [ ]:
import requests
import time
time.sleep(2)

tunnel_info = requests.get("http://localhost:4040/api/tunnels").json()
public_url = tunnel_info["tunnels"][0]["public_url"]
print(f"🌐 Your public URL is: {public_url}")

🌐 Your public URL is: https://6c73-35-197-19-71.ngrok-free.app


In [ ]:
import numpy as np
import cv2
import base64
from ultralytics import YOLO
from flask import Flask, request, jsonify
from io import BytesIO
from PIL import Image
from pyngrok import ngrok
from threading import Thread
import time

app = Flask(__name__)

# ✅ Load YOLOv8 model (pre-trained)
model = YOLO("yolov8n.pt")  # You can swap with yolov8s.pt for better accuracy

@app.route('/detect', methods=['POST'])
def detect_lane():
    try:
        data = request.get_json()
        img_data = base64.b64decode(data['image'])
        img = Image.open(BytesIO(img_data)).convert('RGB')
        img_np = np.array(img)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

        height, width, _ = img_bgr.shape
        center_x = width // 2

        # Detect with YOLO
        results = model(img_bgr)
        result = results[0]
        annotated = result.plot()

        alert_texts = []
        front_distance_alert = False
        left_lane_alert = False
        right_lane_alert = False

        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = result.names[int(box.cls[0])]
            box_center_x = (x1 + x2) // 2

            # Front object detection (check Y proximity)
            if label in ["car", "truck", "bus"] and y2 > height * 0.7:
                alert_texts.append("⚠️ Object in front too close")
                front_distance_alert = True

            # Lane proximity — assume YOLO detects lanes as "lane" class (custom model would help)
            # Check if this object is far left or far right (use box position directly)
            if x1 < width * 0.15:
                alert_texts.append("⚠️ Too close to left lane side")
                left_lane_alert = True
            elif x2 > width * 0.85:
                alert_texts.append("⚠️ Too close to right lane side")
                right_lane_alert = True


        # Draw side bars
        cv2.line(annotated, (int(width * 0.3), 0), (int(width * 0.3), height), (0, 0, 255) if left_lane_alert else (0, 255, 0), 4)
        cv2.line(annotated, (int(width * 0.7), 0), (int(width * 0.7), height), (0, 0, 255) if right_lane_alert else (0, 255, 0), 4)

        _, buffer = cv2.imencode('.jpg', annotated)
        result_b64 = base64.b64encode(buffer).decode('utf-8')

        return jsonify({
            "processed_image": result_b64,
            "alerts": alert_texts
        })
        print("🔁 Sending alerts:", alert_texts)
    except Exception as e:
        return jsonify({"error": str(e)})


"""@app.route('/detect', methods=['POST'])
def detect_lane():
    try:
        # 🔄 Receive image from Swift
        data = request.get_json()
        img_data = base64.b64decode(data['image'])
        img = Image.open(BytesIO(img_data)).convert('RGB')
        img_np = np.array(img)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

        # 🧠 YOLOv8 detection
        results = model(img_bgr)
        result_img = results[0].plot()  # Annotated image

        # 🔁 Encode image and send it back
        _, buffer = cv2.imencode('.jpg', result_img)
        result_b64 = base64.b64encode(buffer).decode('utf-8')

        return jsonify({"processed_image": result_b64})
    except Exception as e:
        return jsonify({"error": str(e)})"""

# 🚀 Launch Flask + pyngrok
def run():
    #import os
    #os.system("fuser -k 5000/tcp")
    app.run()

thread = Thread(target=run)
thread.start()

# Wait for Flask to start
#time.sleep(2)

# Start ngrok tunnel
#public_url = ngrok.connect(5000)
#print(f"Your public URL: {public_url}")


100%|██████████| 6.25M/6.25M [00:00<00:00, 104MB/s]


 * Serving Flask app '__main__'
 * Debug mode: off
